#Agentic RAG: Detailed Guide

This notebook provides a compact, production-minded example of a Agentic Retrieval-Augmented Generation (RAG) pipeline focused on PDF documents. It follows a clear three-agent separation:
- IngestionAgent: Extracts text from PDFs, creates overlapping chunks, and stores both the raw chunks in a local `knowledge_base` and their embeddings in a vector index.
- RetrievalAgent: Performs semantic search against the vector index and assembles human-readable context from the best hits (with a robust fallback to the local `knowledge_base`).
- GenerationAgent: Builds a prompt from the retrieved context and calls an LLM to produce concise, sourced answers.

Quick Start
1. Set `GRAVIXLAYER_API_KEY` in your environment or a `.env` file (the notebook loads `.env`).
2. Update `PDF_PATH` to point to your PDF file or run the sample-data branch in the same cell to populate the demo data.
3. Run the cells top-to-bottom: configuration → vector DB init → ingestion (or sample load) → query examples.
4. Ask questions with `pdf_rag_system.query('Your question here')` — set `show_context=True` to see retrieved snippets.

What’s included
- `VectorDatabase`: Minimal wrapper for GravixLayer vector index operations (create/list/find, upsert, text search).
- `PDFIngestionAgent`: PDF extraction, sentence-aware chunking with overlap, and preparation of payloads for the vector DB and local `knowledge_base`.
- `RetrievalAgent`: Runs text search and assembles readable context; falls back to local `knowledge_base` if the vector search is unavailable or returns limited metadata.
- `GenerationAgent`: Simple LLM prompt wrapper that calls `client.chat.completions.create` and returns a concise answer with source citations when possible.

![PDF RAG Flowchart](images/Agentic_Rag_Flowchart.png)

## 1. Setup and Configuration

In [28]:
# Install required packages
!pip install gravixlayer requests python-dotenv PyPDF2 -q

In [29]:
import os
import json
import time
import re
import requests
import PyPDF2
from typing import List, Dict, Any, Optional
from gravixlayer import GravixLayer
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables
load_dotenv()

# Set up API key
API_KEY = os.getenv('GRAVIXLAYER_API_KEY')
if not API_KEY:
    raise ValueError("Please set GRAVIXLAYER_API_KEY environment variable")

# Initialize GravixLayer client
client = GravixLayer()
print("GravixLayer client initialized successfully")

# Configuration
CONFIG = {
    'embedding_model': 'baai/bge-large-en-v1.5',
    'llm_model': 'meta-llama/llama-3.1-8b-instruct',
    'vector_dimension': 1024,
    'similarity_metric': 'cosine',
    'index_name': 'pdf-rag',
    'top_k_results': 3,
    'base_url': 'https://api.gravixlayer.com/v1/vectors'
}

print("Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

GravixLayer client initialized successfully
Configuration:
  embedding_model: baai/bge-large-en-v1.5
  llm_model: meta-llama/llama-3.1-8b-instruct
  vector_dimension: 1024
  similarity_metric: cosine
  index_name: pdf-rag
  top_k_results: 3
  base_url: https://api.gravixlayer.com/v1/vectors


## 2. Vector Database

In [30]:
# VectorDatabase class - corrected implementation
class VectorDatabase:
    def __init__(self, api_key: str, base_url: str):
        """Initialize the VectorDatabase client with API key and base URL."""
        self.api_key = api_key
        self.base_url = base_url.rstrip('/')
        self.headers = {
            'Authorization': f'Bearer {api_key}',
            'Content-Type': 'application/json'
        }
        self.index_id = None

    def create_index(self, name: str, dimension: int, metric: str) -> Optional[str]:
        """Create a vector index and return its ID if successful."""
        url = f"{self.base_url}/indexes"
        payload = {"name": name, "dimension": dimension, "metric": metric, "vector_type": "dense"}
        resp = requests.post(url, headers=self.headers, json=payload)
        if resp.status_code in (200, 201):
            data = resp.json()
            idx = data.get('id') or data.get('index_id')
            self.index_id = idx
            print(f"Index created: {idx}")
            return idx
        else:
            print(f"Error creating index ({resp.status_code}): {resp.text}")
            return None

    def list_indexes(self) -> list:
        """Return list of indexes (may be empty)."""
        url = f"{self.base_url}/indexes"
        resp = requests.get(url, headers=self.headers)
        if resp.status_code == 200:
            return resp.json().get('indexes', [])
        return []

    def find_or_create_index(self, name: str, dimension: int, metric: str) -> Optional[str]:
        """Return existing index id by name or create a new one."""
        indexes = self.list_indexes()
        for idx in indexes:
            if idx.get('name') == name:
                self.index_id = idx.get('id')
                print(f"Using existing index: {self.index_id}")
                return self.index_id
        return self.create_index(name, dimension, metric)

    def upsert_text_vectors(self, texts_with_metadata: List[Dict]) -> bool:
        """Upsert text vectors into the index. If a 400 occurs persist payload for inspection."""
        if not self.index_id:
            print("No index selected")
            return False
        url = f"{self.base_url}/{self.index_id}/text/upsert"
        payload = {"vectors": texts_with_metadata}
        resp = requests.post(url, headers=self.headers, json=payload)
        if resp.status_code in (200, 201):
            data = resp.json()
            print(f"Upsert successful: {data.get('upserted_count', 'unknown')} vectors")
            return True
        else:
            print(f"Upsert failed ({resp.status_code}): {resp.text}")
            if resp.status_code == 400:
                # Save payload for offline inspection
                try:
                    dump_path = os.path.join(os.getcwd(), 'failed_upsert_payload.json')
                    with open(dump_path, 'w') as f:
                        json.dump({'url': url, 'payload': payload, 'response': resp.text}, f, indent=2)
                    print(f"Saved failing payload to: {dump_path}")
                except Exception as e:
                    print(f"Failed to save payload: {str(e)}")
            return False

    def search_text(self, query: str, model: str, top_k: int = 3, filter_dict: Dict = None) -> List[Dict]:
        """Search the index using a text query and return hits."""
        if not self.index_id:
            print("No index selected")
            return []
        url = f"{self.base_url}/{self.index_id}/search/text"
        payload = {"query": query, "model": model, "top_k": top_k, "include_metadata": True, "include_values": False}
        if filter_dict:
            payload['filter'] = filter_dict
        resp = requests.post(url, headers=self.headers, json=payload)
        if resp.status_code == 200:
            data = resp.json()
            hits = data.get('hits', [])
            print(f"Found {len(hits)} relevant documents (took {data.get('query_time_ms', 'N/A')}ms)")
            return hits
        else:
            print(f"Error searching ({resp.status_code}): {resp.text}")
            return []

# Initialize vector database
vector_db = VectorDatabase(API_KEY, CONFIG['base_url'])
print("Vector database client initialized")

Vector database client initialized


## 3. Knowledge Base

In [31]:
# Global knowledge base
knowledge_base = []  # This will store PDF chunks for retrieval

print("Knowledge base initialized")

Knowledge base initialized


## 4. PDF Processing Agents

In [32]:
# PDF Ingestion Agent
class PDFIngestionAgent:
    def __init__(self, vector_db: VectorDatabase, config: Dict):
        self.vector_db = vector_db
        self.config = config
        
    def ingest_pdf(self, pdf_path: str, custom_name: Optional[str] = None) -> bool:
        """Ingest PDF and add to both vector DB and knowledge base"""
        global knowledge_base  # Use the global knowledge_base like your working RAG
        
        if not os.path.exists(pdf_path):
            print(f"PDF file not found: {pdf_path}")
            return False
        
        if not pdf_path.lower().endswith('.pdf'):
            print(f"File is not a PDF: {pdf_path}")
            return False
        
        source_name = custom_name or os.path.basename(pdf_path)
        display(Markdown(f"### Ingesting PDF: `{source_name}`"))
        
        try:
            # Extract text from PDF
            print("Extracting text from PDF...")
            text = self._extract_pdf_text(pdf_path)
            
            if not text:
                print("No text extracted from PDF")
                return False
            
            print(f"Extracted {len(text)} characters")
            
            # Create chunks
            print("Creating text chunks...")
            chunks = self._create_chunks(text)
            
            if not chunks:
                print("No valid chunks created")
                return False
            
            print(f"Created {len(chunks)} chunks")
            
            # Prepare for vector database
            vectors_for_db = []
            new_knowledge_entries = []
            
            for i, chunk in enumerate(chunks):
                chunk_id = f"{source_name}-chunk-{i}"
                
                # For vector database
                vector_entry = {
                    "id": chunk_id,
                    "text": chunk,
                    "model": self.config['embedding_model'],
                    "metadata": {
                        "source": source_name,
                        "chunk_number": i,
                        "document_type": "pdf"
                    }
                }
                vectors_for_db.append(vector_entry)
                
                # For knowledge base
                knowledge_entry = {
                    "id": chunk_id,
                    "text": chunk,
                    "model": self.config['embedding_model'],
                    "metadata": {
                        "source": source_name,
                        "chunk_number": i,
                        "document_type": "pdf"
                    }
                }
                new_knowledge_entries.append(knowledge_entry)
            
            # Store in vector database
            print("Storing in vector database...")
            success = self.vector_db.upsert_text_vectors(vectors_for_db)
            
            if success:
                # Add to knowledge base
                knowledge_base.extend(new_knowledge_entries)
                print(f"Added {len(new_knowledge_entries)} chunks to knowledge base")
                print(f"Total knowledge base size: {len(knowledge_base)} documents")
                return True
            else:
                print("Failed to store in vector database")
                return False
                
        except Exception as e:
            print(f"Error processing PDF: {str(e)}")
            return False
    
    def _extract_pdf_text(self, pdf_path: str) -> str:
        """Extract text from PDF file"""
        text = ""
        
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                total_pages = len(pdf_reader.pages)
                print(f"PDF has {total_pages} pages")
                
                for i, page in enumerate(pdf_reader.pages, 1):
                    try:
                        page_text = page.extract_text()
                        if page_text.strip():
                            cleaned_text = re.sub(r'\s+', ' ', page_text.strip())
                            text += cleaned_text + " "
                            print(f"  Page {i}: {len(cleaned_text)} characters")
                        else:
                            print(f"  Page {i}: No readable text")
                    except Exception as page_error:
                        print(f"  Error on page {i}: {str(page_error)}")
                        continue
        
        except Exception as e:
            print(f"Error reading PDF: {str(e)}")
            return ""
        
        return text.strip()
    
    def _create_chunks(self, text: str, chunk_size: int = 800, overlap: int = 100) -> List[str]:
        """Create text chunks with overlap"""
        if len(text) < 50:
            return [text] if text.strip() else []
        
        # Split by sentences
        sentences = re.split(r'(?<=[.!?])\s+', text)
        
        chunks = []
        current_chunk = ""
        
        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue
            
            if len(current_chunk) + len(sentence) + 1 > chunk_size and current_chunk:
                if len(current_chunk) >= 50:
                    chunks.append(current_chunk.strip())
                
                # Create overlap
                words = current_chunk.split()
                overlap_words = words[-overlap//10:] if len(words) > overlap//10 else []
                current_chunk = " ".join(overlap_words) + " " + sentence if overlap_words else sentence
            else:
                current_chunk += (" " + sentence) if current_chunk else sentence
        
        # Add final chunk
        if current_chunk.strip() and len(current_chunk) >= 50:
            chunks.append(current_chunk.strip())
        
        return [chunk for chunk in chunks if len(chunk) >= 50 and len(chunk.split()) >= 5]

print("PDF Ingestion Agent ready")

PDF Ingestion Agent ready


In [33]:
# RetrievalAgent and GenerationAgent for 3-agent RAG pattern
class RetrievalAgent:
    def __init__(self, vector_db, knowledge_base, config):
        self.vector_db = vector_db
        self.knowledge_base = knowledge_base
        self.config = config

    def retrieve(self, query: str, top_k: int = None, filter_dict: dict = None):
        """Run vector search and assemble a context string from hits.

        Returns a tuple: (hits, context_str)
        """
        if top_k is None:
            top_k = self.config.get('top_k_results', 3)

        hits = self.vector_db.search_text(
            query=query,
            model=self.config['embedding_model'],
            top_k=top_k,
            filter_dict=filter_dict,
        )

        context_parts = []
        for i, hit in enumerate(hits, 1):
            metadata = hit.get('metadata', {}) or {}
            # Try to find the full text in the local knowledge base
            doc_text = None
            for doc in self.knowledge_base:
                if doc.get('id') == hit.get('id'):
                    doc_text = doc.get('text')
                    break

            # Fall back to any text in metadata
            if not doc_text:
                doc_text = metadata.get('text') or metadata.get('snippet') or ''

            if doc_text:
                source = metadata.get('source', 'unknown')
                chunk_num = metadata.get('chunk_number', 'unknown')
                context_parts.append(
                    f"Source {i} (Score: {hit.get('score', 0):.3f}, File: {source}, Chunk: {chunk_num}):\n{doc_text}"
                )

        context = "\n\n".join(context_parts)
        return hits, context


class GenerationAgent:
    def __init__(self, llm_client, config: dict):
        self.llm_client = llm_client
        self.config = config

    def generate(self, query: str, context: str) -> str:
        """Generate an answer using the LLM client and provided context."""
        prompt = f"""You are a helpful AI assistant. Use the provided context to answer the user's question accurately and comprehensively.

Context:\n{context}\n\nUser Question: {query}\n\nInstructions:\n1. Base your answer primarily on the provided context\n2. If the context doesn't contain enough information, say so clearly\n3. Be concise but thorough\n4. Cite which sources you're using when relevant\n\nAnswer:"""

        try:
            response = self.llm_client.chat.completions.create(
                model=self.config['llm_model'],
                messages=[{"role": "user", "content": prompt}],
                max_tokens=500,
                temperature=0.0,
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error generating response: {str(e)}"

print("✅ RetrievalAgent and GenerationAgent defined")


✅ RetrievalAgent and GenerationAgent defined


In [34]:
# PDF RAG System - orchestrates Ingestion, Retrieval, and Generation agents
class PDFRAGSystem:
    def __init__(self, ingestion_agent: PDFIngestionAgent, retrieval_agent: RetrievalAgent, generation_agent: GenerationAgent, config: Dict):
        """Initialize the PDF RAG system using three agents."""
        self.ingestion_agent = ingestion_agent
        self.retrieval_agent = retrieval_agent
        self.generation_agent = generation_agent
        self.config = config

    def retrieve_context(self, query: str, top_k: int = None, filter_dict: dict = None) -> str:
        """Delegate retrieval to the RetrievalAgent and return context string."""
        _, context = self.retrieval_agent.retrieve(query=query, top_k=top_k, filter_dict=filter_dict)
        return context

    def generate_response(self, query: str, context: str) -> str:
        """Delegate generation to the GenerationAgent."""
        return self.generation_agent.generate(query=query, context=context)

    def query(self, question: str, show_context: bool = True, top_k: int = None, filter_dict: dict = None) -> Dict[str, Any]:
        """High-level method: retrieve context then generate an answer."""
        display(Markdown(f"### 🤔 Query: {question}"))
        print("─" * 80)

        print("🔍 Retrieving relevant context...")
        context = self.retrieve_context(question, top_k=top_k, filter_dict=filter_dict)

        if show_context:
            display(Markdown("#### 📄 Retrieved Context:"))
            print(context)
            print("─" * 80)

        print("🤖 Generating response...")
        response = self.generate_response(question, context)

        display(Markdown("### 💡 Response:"))
        print(response)
        print("═" * 80)

        return {
            'query': question,
            'context': context,
            'response': response
        }

print("✅ PDF RAG System refactored to 3-agent pattern")


✅ PDF RAG System refactored to 3-agent pattern


## 5. System Initialization

In [35]:
# Create or find vector index
index_id = vector_db.find_or_create_index(
    name=CONFIG['index_name'],
    dimension=CONFIG['vector_dimension'],
    metric=CONFIG['similarity_metric']
)

if index_id:
    print(f"📊 Vector index ready: {index_id}")
else:
    print("❌ Failed to create/find vector index")

# Initialize agents
pdf_ingestion = PDFIngestionAgent(vector_db, CONFIG)
retrieval_agent = RetrievalAgent(vector_db, knowledge_base, CONFIG)
# GenerationAgent expects the GravixLayer client; use `client` defined earlier
generation_agent = GenerationAgent(client, CONFIG)

# Create the 3-agent RAG system
pdf_rag_system = PDFRAGSystem(pdf_ingestion, retrieval_agent, generation_agent, CONFIG)

print("✅ PDF RAG System initialized (3-agent pattern)")
display(Markdown("---\n### 🚀 System Ready!"))


Index created: 1390b66f-6cc2-46ec-8d40-8953412091dc
📊 Vector index ready: 1390b66f-6cc2-46ec-8d40-8953412091dc
✅ PDF RAG System initialized (3-agent pattern)


---
### 🚀 System Ready!

## 6. PDF Ingestion

In [36]:
# PDF Ingestion
PDF_PATH = "/Users/rupinajay/Downloads/Test.pdf"  # 📝 UPDATE THIS PATH

if os.path.exists(PDF_PATH):
    print("📥 Ingesting PDF...")
    success = pdf_ingestion.ingest_pdf(PDF_PATH)
    
    if success:
        print(f"\n🎉 PDF ingested successfully!")
        print(f"📚 Knowledge base now has {len(knowledge_base)} documents")
    else:
        print("\n❌ PDF ingestion failed")
else:
    print(f"⚠️ PDF not found: {PDF_PATH}")
    print("Creating sample data for testing...")
    
    # Create sample data for testing
    sample_data = [
        {
            "id": "sample-pdf-chunk-0",
            "text": "This sample PDF document covers artificial intelligence fundamentals, including machine learning algorithms, neural networks, and deep learning techniques. It explains how AI systems learn from data and make predictions based on patterns in training data.",
            "model": CONFIG['embedding_model'],
            "metadata": {
                "source": "sample.pdf",
                "chunk_number": 0,
                "document_type": "pdf"
            }
        },
        {
            "id": "sample-pdf-chunk-1",
            "text": "The document discusses practical AI applications across industries such as healthcare, finance, and transportation. It covers computer vision for image recognition, natural language processing for text analysis, and robotics for automation tasks.",
            "model": CONFIG['embedding_model'],
            "metadata": {
                "source": "sample.pdf",
                "chunk_number": 1,
                "document_type": "pdf"
            }
        }
    ]
    
    # Store in vector database
    success = vector_db.upsert_text_vectors(sample_data)
    
    if success:
        # Add to knowledge base
        knowledge_base.extend(sample_data)
        print(f"Sample data loaded. Knowledge base has {len(knowledge_base)} documents")
    else:
        print("Failed to load sample data")

📥 Ingesting PDF...


### Ingesting PDF: `Test.pdf`

Extracting text from PDF...
PDF has 44 pages
  Page 1: 167 characters
  Page 2: 1191 characters
  Page 3: 2338 characters
  Page 4: 2058 characters
  Page 5: 3257 characters
  Page 6: 2854 characters
  Page 7: 2306 characters
  Page 8: 1771 characters
  Page 9: 3483 characters
  Page 10: 1650 characters
  Page 11: 3067 characters
  Page 12: 2263 characters
  Page 13: 3034 characters
  Page 14: 2539 characters
  Page 15: 2514 characters
  Page 16: 3259 characters
  Page 17: 2135 characters
  Page 18: 2648 characters
  Page 19: 2761 characters
  Page 20: 2029 characters
  Page 21: 995 characters
  Page 22: 3336 characters
  Page 23: 1780 characters
  Page 24: 2540 characters
  Page 25: 1932 characters
  Page 26: 2319 characters
  Page 27: 2101 characters
  Page 28: 2969 characters
  Page 29: 1152 characters
  Page 30: 113 characters
  Page 31: 3647 characters
  Page 32: 1087 characters
  Page 33: 4096 characters
  Page 34: 1887 characters
  Page 35: 2455 characters
  Page 36: 2814 charac

## 7. Query Your PDF

In [37]:
# Test Query 1
result1 = pdf_rag_system.query("What is the main topic of the PDF document?")

### 🤔 Query: What is the main topic of the PDF document?

────────────────────────────────────────────────────────────────────────────────
🔍 Retrieving relevant context...
Found 3 relevant documents (took 54ms)
Found 3 relevant documents (took 54ms)


#### 📄 Retrieved Context:

Source 1 (Score: 0.574, File: Test.pdf, Chunk: 105):
conversational way).Add a column including the month they were published. AIReport Title Description Month Published Economic Impact ResearchThis report provides an in -depth analysis of the economic impact of the travel and tourism industry, both globally and in individual countries.

Source 2 (Score: 0.570, File: Test.pdf, Chunk: 152):
© World Travel & Tourism Council: Introduction to AI 2024. All rights reserved. The copyright laws of the United Kingdom allow certain uses of this content without our (i.e. the copyright owner’s) permission. You are permitted to use limited extracts of this content, provided such use is fair and when such use is for non-commercial research, private study, review or news reporting. The following acknowledgment must also be used, whenever our content is used relying on this “fair dealing” exception: “Source: World Travel and Tourism Council: Introduction to AI 2024.

Source 3 (Score: 0.563, File: Test

### 💡 Response:

Based on the provided context, it appears that the main topic of the PDF document is related to the travel and tourism industry, specifically economic impact research and analysis.

From Source 1 (Score: 0.574), we can see that the report provides an "in-depth analysis of the economic impact of the travel and tourism industry" (Source 1).

Similarly, from Source 3 (Score: 0.563) and Source 2 does not mention a main topic but rather provides context for permitted uses of the content, it is clear that the PDF document discusses various topics related to the travel and tourism industry, including economic impact research.

Therefore, I can conclude that the main topic of the PDF document is likely "Travel and Tourism Industry Analysis" with a focus on economic impact. 

Sources used:
- Source 1: Score: 0.574
- Source 2 does not provide information but mentions context for permitted uses.
- Source 3
════════════════════════════════════════════════════════════════════════════════


In [38]:
# Test Query 2
result2 = pdf_rag_system.query("What AI applications are mentioned?")

### 🤔 Query: What AI applications are mentioned?

────────────────────────────────────────────────────────────────────────────────
🔍 Retrieving relevant context...
Found 3 relevant documents (took 61ms)
Found 3 relevant documents (took 61ms)


#### 📄 Retrieved Context:

Source 1 (Score: 0.704, File: Test.pdf, Chunk: 79):
to classify an AI system is based on its functionality. For many years this involved an AI solution either being considered as an Expert System, or as Predictive AI, but now also includes a relatively new field of artificial intelligence functionality called Generative AI. An AI Expert System is a smart computer program that uses AI to simulate the expertise of humans in a specific area, such as for travel planning, medical diagnosis or financial advice. Expert systems contain a knowledge base of rules and facts about a specific domain, which are combined with AI (in a system called an ‘inference engine’) to reason through problems and provide advice, or recommendations. For example a travel AI expert system could help travellers to plan and book their trips.

Source 2 (Score: 0.701, File: Test.pdf, Chunk: 52):
products, content, or services tailored to an individual user. 6. Natural Language Processing (NLP): NLP models, such as AI p

### 💡 Response:

Several AI applications are mentioned in the provided context:

1. **Expert Systems** (Source 1): These are AI systems that simulate human expertise in specific areas such as travel planning, medical diagnosis, or financial advice.
2. **Predictive AI** (Source 1): Although not explicitly described in detail, it is mentioned as a category of AI functionality separate from Expert Systems and Generative AI.
3. **Generative AI** (Source 1): A relatively new field that refers to AI functionality where the system can generate novel content or outputs.
4. **Natural Language Processing (NLP) models**: These include AI powered chatbots, virtual assistants, and language translation systems that benefit from diverse language data (Source 2).
5. **Real-Time Decision Making** (Source 2): This refers to AI systems that can process and analyze big data in real-time to provide instant recommendations or decisions.
6. **AI NLP models for understanding and generating human-like language** (Source 2): Th

In [39]:
# Test Query 3
result3 = pdf_rag_system.query("Summarize the key points from the document")

### 🤔 Query: Summarize the key points from the document

────────────────────────────────────────────────────────────────────────────────
🔍 Retrieving relevant context...
Found 3 relevant documents (took 54ms)
Found 3 relevant documents (took 54ms)


#### 📄 Retrieved Context:

Source 1 (Score: 0.550, File: Test.pdf, Chunk: 2):
Global Digital Divide & AI Skills Gap 31 QUIZ ....................................................................................................................... 34 ANNEX : AI TERMINOLOGY .......................................................................... 35 QUIZ ANSWER ................................................................................................... 40 ACKNOWLEDGEMENTS ................................................................................. 41 ENDNOTES ..........................................................................................................

Source 2 (Score: 0.550, File: Test.pdf, Chunk: 105):
conversational way).Add a column including the month they were published. AIReport Title Description Month Published Economic Impact ResearchThis report provides an in -depth analysis of the economic impact of the travel and tourism industry, both globally and in individual countries.

Sourc

### 💡 Response:

Based on the provided context, here's a summary of the key points:

The document appears to be related to Artificial Intelligence (AI), but it doesn't provide a clear and comprehensive overview. However, we can extract some information from the snippets.

* The document mentions a "Global Digital Divide & AI Skills Gap" which suggests that there is a gap in skills related to AI (Source 1) [0].
* It also includes a report on the "Economic Impact of Travel and Tourism Industry" with details on analysis by countries (Source 2).
* Another snippet provides an introduction to Artificial Intelligence, including its history, types, and what drives it (Source 3).

Unfortunately, there isn't enough information in the provided context to provide a more detailed or insightful summary. However, based on the snippets, these appear to be key points.

Sources used:

[0] Source 1: Global Digital Divide & AI Skills Gap
════════════════════════════════════════════════════════════════════════════════


In [40]:
# Test Query 4 - Without showing context
result4 = pdf_rag_system.query("Explain machine learning concepts in the doc")

### 🤔 Query: Explain machine learning concepts in the doc

────────────────────────────────────────────────────────────────────────────────
🔍 Retrieving relevant context...
Found 3 relevant documents (took 61ms)
Found 3 relevant documents (took 61ms)


#### 📄 Retrieved Context:

Source 1 (Score: 0.670, File: Test.pdf, Chunk: 33):
and to turn a ‘black box’ into a ‘glass box’). The ‘digital doctor’ example nicely illustrates why ‘explainable AI’ is so important (in this case for both the patient and the doctor) to understand why an AI system has made a decision and believes a patient has breast cancer before sending them for intrusive cancer treatment. When considering how an AI system is constructed, there are two primary methods for how an AI system ‘learns’ to perform a task, which are machine learning and deep learning. • Machine Learning : Machine Learning (ML) systems can learn from huge amounts of data and continuously improve their performance over time when provided with more and/or better quality training data.

Source 2 (Score: 0.651, File: Test.pdf, Chunk: 131):
could do, such as carry guests’ luggage to their room. Machine LearningMachine learning is a type of AI that involves training algorithms with large amounts of data, so that a computer is cap

### 💡 Response:

According to the provided context, Machine Learning (ML) is a type of AI that involves training algorithms with large amounts of data, allowing a computer to make predictions or decisions [Source 1 & Source 2]. The key concepts mentioned in the context are:

* **Learning from huge amounts of data**: ML systems can improve their performance over time when provided with more and/or better quality training data [Source 1].
* **Training algorithms**: Machine Learning involves training algorithms on a dataset, so that they can learn to make predictions or decisions [Source 2].
* **Examples of machine learning applications**: Email spam filters [Source 2] are an example of machine learning in action.

Note: The provided context does not go into detail about the underlying mechanics or math behind Machine Learning. For more information on this topic, you may need to consult additional sources.
════════════════════════════════════════════════════════════════════════════════


## 8. System Status

In [41]:
# Show system status
print(f"📊 System Status:")
print(f"  - Vector Database: {'✅ Connected' if vector_db.index_id else '❌ Not connected'}")
print(f"  - Knowledge Base Size: {len(knowledge_base)} documents")
print(f"  - Index ID: {vector_db.index_id}")

if knowledge_base:
    print(f"\n📚 Knowledge Base Contents:")
    for i, doc in enumerate(knowledge_base[:3]):  # Show first 3
        source = doc['metadata']['source']
        chunk_num = doc['metadata']['chunk_number']
        text_preview = doc['text'][:100] + '...' if len(doc['text']) > 100 else doc['text']
        print(f"  {i+1}. {source} (Chunk {chunk_num}): {text_preview}")
    
    if len(knowledge_base) > 3:
        print(f"  ... and {len(knowledge_base) - 3} more documents")

📊 System Status:
  - Vector Database: ✅ Connected
  - Knowledge Base Size: 155 documents
  - Index ID: 1390b66f-6cc2-46ec-8d40-8953412091dc

📚 Knowledge Base Contents:
  1. Test.pdf (Chunk 0): INTRODUCTION TO AI World Travel & Tourism Council< Contents | 1 INTRODUCTION TO ARTIFICIAL INTELLIGE...
  2. Test.pdf (Chunk 1): Council2 < Contents | CONTENTS FOREWORD ...............................................................
  3. Test.pdf (Chunk 2): Global Digital Divide & AI Skills Gap 31 QUIZ .........................................................
  ... and 152 more documents
